In [35]:
import os
os.chdir("/Users/n0362948/desktop/Bread")

In [36]:
import pandas as pd
intellicron_prequals=pd.read_csv('/Users/n0362948/desktop/Bread/intellicron_prequals.csv')
prequals=pd.read_csv('/Users/n0362948/desktop/Bread/prequals.csv')

In [37]:
intellicron_prequals['group']='test'

In [38]:
date_min=intellicron_prequals['assignment_date'].min()
date_max=intellicron_prequals['assignment_date'].max()

In [39]:
temp=pd.merge(prequals,intellicron_prequals,how='left', on='prequal_id')

In [40]:
temp['dummy']=1

In [41]:
def group(row):
    if row['group']=='test': 
        return 'test'
    else:
        return 'control'
    
temp['group']=temp.apply(group, axis=1)

In [42]:
split=temp.loc[(temp4['prequal_date']<=date_max) & (temp['prequal_date']>=date_min)]

In [43]:
clean_split=split[['prequal_id','checkout_id','prequal_date','completed_prequal','approved','assignment_date','group','dummy']]

In [47]:
clean_split.shape, clean_split.columns

((18185, 8),
 Index(['prequal_id', 'checkout_id', 'prequal_date', 'completed_prequal',
        'approved', 'assignment_date', 'group', 'dummy'],
       dtype='object'))

In [50]:
clean_split[['completed_prequal', 'approved',  'group', 'dummy']].groupby('group').agg('sum')

,completed_prequal,approved,dummy
group,,,
control,5026,3371,9690
test,4658,3088,8495


In [51]:
import numpy as np
summary=clean_split.pivot_table(values='completed_prequal', index='group', aggfunc=np.sum)
summary['total']=clean_split.pivot_table(values='completed_prequal', index='group', aggfunc=lambda x:len(x))
summary['completion_prequal_rate']=clean_split.pivot_table(values='completed_prequal', index='group')



In [54]:
summary.reset_index()

,group,completed_prequal,total,completion_prequal_rate
0,control,5026,9690,0.518679
1,test,4658,8495,0.548323


In [64]:
from statsmodels.stats.proportion import proportions_ztest
import scipy.stats as stats

In [73]:
def prop_test(convert_control, size_control, convert_test, size_test):
  
    p_a = convert_control / size_control
    p_b = convert_test / size_test
    p_pooled = (convert_control + convert_test) / (size_control + size_test)
    var = p_pooled * (1 - p_pooled) * (1 / size_control + 1 / size_test)
    zscore = np.abs(p_b - p_a) / np.sqrt(var)
    one_side = 1 - stats.norm(loc = 0, scale = 1).cdf(zscore)
    pvalue = one_side * 2
    return zscore, pvalue

In [75]:
### this is data for completed prequal out of all test data--> to calculate p value

convert_control = 5026
size_control = 9690
convert_test = 4658
size_test = 8495

zscore, pvalue = prop_test(convert_control, size_control, convert_test, size_test)
print('zscore = {:.3f}, pvalue = {:.6f}'.format(zscore, pvalue))

zscore = 3.997, pvalue = 0.000064


In [79]:
def confidence_interval(convert_control, size_control, convert_test, size_test, significance = 0.05):
    """
    A/B test for two proportions;
    given a success a trial size of group A and B compute
    its confidence interval;
    resulting confidence interval matches R's prop.test function
    
    Parameters
    ----------
    success_a, success_b : int
        Number of successes in each group
        
    size_a, size_b : int
        Size, or number of observations in each group
        
    significance : float, default 0.05
        Often denoted as alpha. Governs the chance of a false positive.
        A significance level of 0.05 means that there is a 5% chance of
        a false positive. In other words, our confidence level is
        1 - 0.05 = 0.95
        
    Returns
    -------
    prop_diff : float
        Difference between the two proportion
    
    confint : 1d ndarray
        Confidence interval of the two proportion test
    """
    p_a = convert_control / size_control
    p_b = convert_test / size_test
    var = p_a * (1 - p_a) / convert_control + p_b * (1 - p_b) / size_test
    se = np.sqrt(var)
    
    # z critical value
    confidence = 1 - significance
    z = stats.norm(loc = 0, scale = 1).ppf(confidence + significance / 2)

    # standard formula for the confidence interval
    # point-estimtate +- z * standard-error
    prop_diff = p_b - p_a
    confint = prop_diff + np.array([-1, 1]) * z * se
    return prop_diff, confint

In [80]:
### this is data for completed prequal out of all test data--> to calculate confidence interval

prop_diff, confint = confidence_interval(convert_control, size_control, convert_test, size_test)
print('estimate difference:', prop_diff)
print('confidence interval:', confint)

estimate difference: 0.029643492104564628
confidence interval: [ 0.01224213  0.04704485]


In [ ]:

summary_2=clean_split.pivot_table(values='completed_prequal', index='group', aggfunc=np.sum)
summary_2['total']=clean_split.pivot_table(values='completed_prequal', index='group', aggfunc=lambda x:len(x))
summary_['completion_prequal_rate']=clean_split.pivot_table(values='completed_prequal', index='group')

In [84]:
### this is data for approved out of all test data--> to calculate p value and confidence interval 

convert_control2 = 3371
size_control2 = 9690
convert_test2 = 3088
size_test2 = 8495

zscore, pvalue = prop_test(convert_control2, size_control2, convert_test2, size_test2)
print('zscore = {:.3f}, pvalue = {:.6f}'.format(zscore, pvalue))

zscore = 2.196, pvalue = 0.028060


In [85]:
prop_diff, confint = confidence_interval(convert_control2, size_control2, convert_test2, size_test2)
print('estimate difference:', prop_diff)
print('confidence interval:', confint)

estimate difference: 0.015623528925835684
confidence interval: [-0.00343291  0.03467996]
